In [2]:
import pandas as pd
import numpy as np

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:


In [70]:
#Read Postal Codes of Canada into dataframe

df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') 
df =df[0]

df.head()
 

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [71]:
df.set_index("Postal Code" , inplace=True)
df.head()
 

,Borough,Neighborhood
Postal Code,,
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"


More than one neighborhood can exist in one postal code area.

In [67]:
#Drop Neighborhood is no assigned

df=df[df["Neighborhood"]!='Not assigned']
df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [68]:
#reemplazar values

import numpy as np
df['Borough'] = np.where(df['Borough'] =="Not assigned", df['Neighborhood'], df['Borough'])
df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [72]:
df.shape

(180, 2)